In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder


In [2]:
df=pd.read_csv(r"C:\Users\kirth\Downloads\energy-pop-exposure-nuclear-plants-locations_plants.csv")

In [3]:
df.dropna(axis='columns', inplace=True)

In [4]:
df

,FID,Country,Plant,NumReactor,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,0,SWEDEN,AGESTA,1,59.206022,18.082872,187382000,188684000,188250000,121178000.0,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,1,SPAIN,ALMARAZ,2,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,2,BRAZIL,ANGRA,3,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,3,UNITED STATES OF AMERICA,ARKANSAS ONE,2,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,4,SPAIN,ASCO,2,41.200000,0.566670,271854000,287134000,308922000,190825000.0,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,CHINA,YANGJIANG,3,21.708333,112.261111,557032000,629419000,679747000,250848000.0,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,272,UNITED STATES OF AMERICA,YANKEE,1,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,273,"KOREA, REPUBLIC OF",YONGGWANG,6,35.411130,126.416190,649425000,710033000,750775000,404604000.0,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,274,UKRAINE,ZAPOROZHE,6,47.511809,34.585460,273432000,274834000,272561000,162035000.0,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [5]:
columns_to_drop = ['FID', 'Plant', 'NumReactor']
df.drop(columns=columns_to_drop, inplace=True)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [6]:

# Label Encoding
label_encoder = LabelEncoder()  # Create a LabelEncoder instance
categorical_cols = [ 'Country']  # Specify the categorical columns to be encoded

for col in categorical_cols:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])  # Apply label encoding to each column

print("Label Encoded Data:")
df

Label Encoded Data:


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          276 non-null    object 
 1   Latitude         276 non-null    float64
 2   Longitude        276 non-null    float64
 3   p90_1200         276 non-null    int64  
 4   p00_1200         276 non-null    int64  
 5   p10_1200         276 non-null    int64  
 6   p90u_1200        276 non-null    float64
 7   p00u_1200        276 non-null    float64
 8   p10u_1200        276 non-null    float64
 9   p90r_1200        276 non-null    int64  
 10  p00r_1200        276 non-null    int64  
 11  p10r_1200        276 non-null    int64  
 12  p90_30           276 non-null    float64
 13  p00_30           276 non-null    float64
 14  p10_30           276 non-null    float64
 15  p90u_30          276 non-null    float64
 16  p00u_30          276 non-null    float64
 17  p10u_30         

In [8]:
X=df[["p90_1200","p00_1200","p10_1200"]]
X

,p90_1200,p00_1200,p10_1200
0,187382000,188684000,188250000
1,136675000,147718000,163429000
2,99195200,113894000,127898000
3,117830000,132729000,146482000
4,271854000,287134000,308922000
...,...,...,...
271,557032000,629419000,679747000
272,123430000,131527000,145162000
273,649425000,710033000,750775000
274,273432000,274834000,272561000


In [9]:
import numpy as np
import pandas as pd

def estimate_coef(x, y):
    # number of observations/points
    n = np.size(x)

    # mean of x and y vector
    m_x = np.mean(x)
    m_y = np.mean(y)

    # calculating cross-deviation and deviation about x
    SS_xy = np.sum(y*x) - n*m_y*m_x
    SS_xx = np.sum(x*x) - n*m_x*m_x

    # calculating regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1*m_x

    return (b_0, b_1)

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

M = []
N = []

for i in range(0, 276):
    x = np.array([0.1, 0.15, 0.20])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])
    coef = estimate_coef(x, y)
        # Accessing the regression coefficients
    b_0, b_1 = coef
    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    M.append(b_0)
    N.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8679999.999999968
Regression Coefficients for row 1: b_0 = 109143000.00000015, b_1 = 267539999.99999902
Regression Coefficients for row 2: b_0 = 70608200.00000016, b_1 = 287027999.9999989
Regression Coefficients for row 3: b_0 = 89369000.00000016, b_1 = 286519999.9999989
Regression Coefficients for row 4: b_0 = 233701333.33333352, b_1 = 370679999.9999986
Regression Coefficients for row 5: b_0 = 44300533.33333342, b_1 = 161239999.9999994
Regression Coefficients for row 6: b_0 = 354666833.33333343, b_1 = 226729999.99999917
Regression Coefficients for row 7: b_0 = 146591666.66666663, b_1 = -80039999.9999997
Regression Coefficients for row 8: b_0 = 334174000.00000006, b_1 = 126459999.99999954
Regression Coefficients for row 9: b_0 = 135709000.00000018, b_1 = 332979999.9999988
Regression Coefficients for row 10: b_0 = 285644666.66666675, b_1 = 79879999.9999997
Regression Coefficients for row 11: b_0 = 329247833.3333335, b_1 

In [10]:
p20_1200=[]
for j in range(0,276):
    y=M[j]+N[j]*0.25
    p20_1200.append(y)

In [11]:
p20_1200=np.fix(p20_1200)
p20_1200

array([1.88973333e+08, 1.76027999e+08, 1.42365199e+08, 1.60998999e+08,
       3.26371333e+08, 8.46105330e+07, 4.11349333e+08, 1.26581666e+08,
       3.65788999e+08, 2.18953999e+08, 3.05614666e+08, 3.96635333e+08,
       5.38374000e+07, 3.05137999e+08, 4.35906666e+08, 4.33829999e+08,
       1.98190333e+08, 2.76039000e+05, 3.76270333e+08, 1.54162666e+08,
       4.15294000e+08, 7.35279330e+07, 3.80831666e+08, 3.45151999e+08,
       2.03938999e+08, 3.11338999e+08, 3.96490999e+08, 2.06890333e+08,
       2.04790333e+08, 3.94534666e+08, 1.93838666e+08, 4.41982666e+08,
       1.55628866e+08, 1.76452666e+08, 1.81855666e+08, 2.04021333e+08,
       4.53056999e+08, 2.23660333e+08, 4.18998666e+08, 8.97854333e+08,
       2.94826333e+08, 6.20960333e+08, 2.43572666e+08, 7.68232333e+08,
       2.71011333e+08, 7.03092666e+08, 3.64281333e+08, 4.00614333e+08,
       3.87241999e+08, 2.01744999e+08, 2.32434666e+08, 4.70343660e+07,
       1.02384733e+08, 1.36467999e+08, 4.43616666e+08, 4.39727333e+08,
      

In [12]:
df['p20_1200LR'] = p20_1200.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

A = []
B = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    # Create a linear regression model
    model = LinearRegression()
    model.fit(x, y)

    # Access the regression coefficients
    b_0 = model.intercept_
    b_1 = model.coef_[0]

    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    A.append(b_0)
    B.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8680000.0
Regression Coefficients for row 1: b_0 = 109142999.99999999, b_1 = 267540000.00000012
Regression Coefficients for row 2: b_0 = 70608200.0, b_1 = 287028000.0
Regression Coefficients for row 3: b_0 = 89368999.99999999, b_1 = 286520000.0000001
Regression Coefficients for row 4: b_0 = 233701333.3333333, b_1 = 370680000.00000006
Regression Coefficients for row 5: b_0 = 44300533.33333332, b_1 = 161240000.00000006
Regression Coefficients for row 6: b_0 = 354666833.3333333, b_1 = 226730000.00000006
Regression Coefficients for row 7: b_0 = 146591666.66666666, b_1 = -80040000.00000001
Regression Coefficients for row 8: b_0 = 334174000.0, b_1 = 126460000.00000001
Regression Coefficients for row 9: b_0 = 135709000.0, b_1 = 332980000.00000006
Regression Coefficients for row 10: b_0 = 285644666.6666667, b_1 = 79880000.00000003
Regression Coefficients for row 11: b_0 = 329247833.3333333, b_1 = 269550000.00000006
Regression C

Regression Coefficients for row 147: b_0 = 478332166.6666666, b_1 = 1429430000.0000002
Regression Coefficients for row 148: b_0 = 345573833.3333333, b_1 = 205070000.00000003
Regression Coefficients for row 149: b_0 = 149662166.66666666, b_1 = -66710000.00000001
Regression Coefficients for row 150: b_0 = 378363333.3333333, b_1 = 297160000.00000006
Regression Coefficients for row 151: b_0 = 381111666.66666657, b_1 = 1166220000.0000005
Regression Coefficients for row 152: b_0 = 302657833.33333325, b_1 = 1081930000.0000002
Regression Coefficients for row 153: b_0 = 222541166.66666666, b_1 = 1334290000.0000002
Regression Coefficients for row 154: b_0 = 83659333.33333333, b_1 = 169640000.00000003
Regression Coefficients for row 155: b_0 = 346640666.6666667, b_1 = 364680000.0000001
Regression Coefficients for row 156: b_0 = 135896000.0, b_1 = 363960000.00000006
Regression Coefficients for row 157: b_0 = 124231833.33333333, b_1 = 501490000.0000001
Regression Coefficients for row 158: b_0 = 187

In [14]:
p20_1200LIB=[]
for j in range(0,276):
    y=A[j]+B[j]*0.25
    p20_1200LIB.append(y)
p20_1200LIB=np.fix(p20_1200LIB)
p20_1200LIB
#USING LIBRARY IS LIB
df['p20_1200LIB'] = p20_1200LIB.tolist()
df


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0


In [15]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

D = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])
    
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(x, y)
    y_pred = regressor.predict([[0.25]])
    D.append(y_pred)

In [16]:
D=np.fix(D)
df['P20_1200DT'] = D.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,[188250000.0]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,[163429000.0]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,[127898000.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,[146482000.0]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,[308922000.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,[679747000.0]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,[145162000.0]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,[750775000.0]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,[272561000.0]


In [17]:
df.loc[:, 'P20_1200DT'] = D.tolist()


C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\512726359.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200DT'] = D.tolist()


In [18]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0


In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

s = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])
    
    # Create a Ridge regression model
    regressor = Ridge(alpha=1.0)
    regressor.fit(x, y)
    
    x_test = np.array([[0.25]])
    y_pred = regressor.predict(x_test)
    s.append(y_pred)


In [20]:
df['P20_1200ridge'] = s
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,[188109651.74129355]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,[149407104.47761193]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,[113805200.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,[132489547.26368159]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,[289487751.2437811]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,[622676522.3880597]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,[133481119.40298507]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,[703915228.8557214]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,[273604666.6666667]


In [21]:
df.loc[:, 'P20_1200ridge'] = s


C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3074586067.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200ridge'] = s


In [22]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08


In [23]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor



# Create a new column 'Predicted_p20_1200' and set it to 0
df['Predicted_p20_1200'] = 0

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create a KNN regressor object
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the model to the data
knn.fit(X, y)

# Make predictions for the input features
predictions = knn.predict(X)

# Assign the predicted values to the 'Predicted_p20_1200knn' column
df['Predicted_p20_1200knn'] = predictions


In [24]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,0,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,0,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,0,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,0,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,0,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,0,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,0,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,0,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,0,281498999.8


In [25]:
# Drop the 'knn2' and 'Predicted_p20_1200' columns
df = df.drop([ 'Predicted_p20_1200'], axis=1)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8


In [26]:
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create an ANN regressor object
regressor = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)

# Fit the model to the data
regressor.fit(X, y)

# Make predictions for the input features
predictions = regressor.predict(X)

# Add the predictions as a new column in the DataFrame
df['ANN2'] = predictions


In [27]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08


In [28]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values
y = df['p20_1200LR'].values

# Perform feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create an ElasticNet regressor object
regressor = ElasticNet(alpha=0.5, l1_ratio=0.5)

# Fit the model to the scaled data
regressor.fit(X_scaled, y)

# Make predictions for the scaled data
predictions = regressor.predict(X_scaled)

# Add the predictions as a new column in the DataFrame
df['en'] = predictions


In [29]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [30]:
df = df.drop('Country', axis=1)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,66384500,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,53774400,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,36072700,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,27947300,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,86668900,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,341528000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,21426000,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,269049000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,111341000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [31]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming 'df' is your dataset

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize each column in the dataset
for column in df.columns:
    # Reshape the data to 2D array before scaling
    column_data = df[column].values.reshape(-1, 1)
    df[column] = scaler.fit_transform(column_data)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,0.913304,0.489398,0.226290,0.187065,0.158115,0.249432,0.223774,0.210121,0.109688,0.090250,...,0.015467,0.013903,0.848485,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,0.723163,0.407610,0.164938,0.146380,0.137234,0.180179,0.171884,0.178023,0.081266,0.073026,...,0.065309,0.062053,0.818182,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,0.107433,0.274295,0.119590,0.112788,0.107343,0.139440,0.142382,0.150086,0.051805,0.048847,...,0.073693,0.070927,0.090909,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,0.679075,0.106545,0.142137,0.131494,0.122977,0.190616,0.191717,0.198614,0.041441,0.037749,...,0.019304,0.018286,1.000000,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,0.736806,0.429153,0.328496,0.284839,0.259634,0.392819,0.366816,0.369266,0.134377,0.117956,...,0.031856,0.030315,0.818182,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,0.545747,0.813314,0.673545,0.624773,0.571601,0.516392,0.526803,0.531251,0.509313,0.466063,...,0.326416,0.299600,0.181818,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,0.751782,0.176372,0.148912,0.130300,0.121866,0.212784,0.201452,0.208687,0.032836,0.028842,...,0.035898,0.033932,1.000000,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.680063,0.861999,0.785335,0.704834,0.631356,0.832940,0.806956,0.798312,0.407129,0.367065,...,0.064481,0.057768,0.515152,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,0.798676,0.546157,0.330406,0.272623,0.229045,0.333547,0.299158,0.278708,0.184946,0.151655,...,0.061535,0.048980,0.939394,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [32]:
from sklearn.decomposition import PCA

# Separate features and target variables
target_columns = ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']
features = df.drop(target_columns, axis=1)
target_variables = df[target_columns]

# Apply dimensionality reduction using PCA
pca = PCA(n_components=5)  # Set the desired number of components
features_reduced = pca.fit_transform(features)

# Create a new DataFrame for the reduced features
df_reduced = pd.DataFrame(features_reduced, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# Concatenate the reduced features with the target variables
df_combined = pd.concat([df_reduced, target_variables], axis=1)


In [33]:
df_combined

,PC1,PC2,PC3,PC4,PC5,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,-0.802412,0.070108,0.533996,-0.213804,-0.027186,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,-0.809580,0.159435,-0.019536,0.127898,0.036270,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,-0.034519,-0.280523,-0.363478,0.023808,0.644185,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,-1.078497,0.039058,0.073401,0.177846,-0.135125,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,-0.501050,0.171726,0.019281,0.036630,-0.029960,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...
271,1.232798,0.643078,-0.294524,0.178935,0.096370,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,-0.369139,-0.368610,-0.332430,0.160723,-0.245654,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.767564,0.617158,-0.073923,-0.267815,-0.203559,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,-0.478618,0.140030,0.142593,0.061935,-0.085718,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [34]:
# Define the number of folds for cross-validation
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True)
# Assuming you have multiple target variables named 'target_variable1', 'target_variable2', etc.
target_variables =  ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']

In [35]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Define the number of folds for cross-validation
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=0.2,
                                                    random_state=42)

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
param_grid = {
    'fit_intercept': [True, False],
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the linear regression model
    model = LinearRegression()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        rmse_val = np.sqrt(mean_squared_error(y_val.iloc[:, i], y_pred_val[:, i]))
        mae_val = mean_absolute_error(y_val.iloc[:, i], y_pred_val[:, i])
        r2_val = r2_score(y_val.iloc[:, i], y_pred_val[:, i])
        explained_variance_val = explained_variance_score(y_val.iloc[:, i], y_pred_val[:, i])

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': rmse_val,
            'MAE': mae_val,
            'R2 Score': r2_val,
            'Explained Variance': explained_variance_val
        }, ignore_index=True)

# Print the error DataFrame in a tabular format
print(error_df)

# Train the model with the best parameters on the full training set
best_model.fit(X_train, y_train)



C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWar

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWar

          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.032768  0.027241  0.900124            0.900892
1             p20_1200LIB  0.032768  0.027241  0.900124            0.900892
2              P20_1200DT  0.029043  0.024496  0.931467            0.931626
3           P20_1200ridge  0.031562  0.026921  0.934481            0.934482
4   Predicted_p20_1200knn  0.034250  0.028351  0.913680            0.915214
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.035432  0.026196  0.942200            0.944216
66          P20_1200ridge  0.039264  0.029536  0.940087            0.944606
67  Predicted_p20_1200knn  0.042603  0.030549  0.925516            0.928315
68                   ANN2  0.039573  0.029883  0.940509            0.945227
69                     en  0.039028  0.029332  0.940178            0.944526

[70 rows x 5 columns]


C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\881131965.py:52: FutureWar

LinearRegression()

In [36]:
# Print the error DataFrame in a tabular format
print(error_df)

# Calculate the mean error
mean_rmse = error_df['RMSE'].mean()
mean_mae = error_df['MAE'].mean()
mean_r2 = error_df['R2 Score'].mean()
mean_explained_variance = error_df['Explained Variance'].mean()

print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.032768  0.027241  0.900124            0.900892
1             p20_1200LIB  0.032768  0.027241  0.900124            0.900892
2              P20_1200DT  0.029043  0.024496  0.931467            0.931626
3           P20_1200ridge  0.031562  0.026921  0.934481            0.934482
4   Predicted_p20_1200knn  0.034250  0.028351  0.913680            0.915214
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.035432  0.026196  0.942200            0.944216
66          P20_1200ridge  0.039264  0.029536  0.940087            0.944606
67  Predicted_p20_1200knn  0.042603  0.030549  0.925516            0.928315
68                   ANN2  0.039573  0.029883  0.940509            0.945227
69                     en  0.039028  0.029332  0.940178            0.944526

[70 rows x 5 columns]
Mean RMSE: 0.03563420001207383
Mean MAE: 0.026285285872568145
Mea

In [37]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
lasso_rmse_list = []
lasso_mae_list = []
lasso_r2_list = []
lasso_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'max_iter': [100, 500, 1000]  # Adjust the values of max_iter as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Lasso regression model
    lasso_model = Lasso()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(lasso_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_lasso_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_lasso_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        lasso_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        lasso_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        lasso_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        lasso_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        lasso_rmse_list.append(lasso_rmse_val)
        lasso_mae_list.append(lasso_mae_val)
        lasso_r2_list.append(lasso_r2_val)
        lasso_explained_variance_list.append(lasso_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': lasso_rmse_val,
            'MAE': lasso_mae_val,
            'R2 Score': lasso_r2_val,
            'Explained Variance': lasso_explained_variance_val
        }, ignore_index=True)

        print("Lasso RMSE for", target_variable, ":", lasso_rmse_val)
        print("Lasso MAE for", target_variable, ":", lasso_mae_val)
        print("Lasso R2 score for", target_variable, ":", lasso_r2_val)
        print("Lasso Explained variance score for", target_variable, ":", lasso_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)

# Train the model with the best parameters on the full training set
best_lasso_model.fit(X_train, y_train)


C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: Futu

Lasso RMSE for p20_1200LR : 0.126918100299615
Lasso MAE for p20_1200LR : 0.10278259020143371
Lasso R2 score for p20_1200LR : 0.17237942719416488
Lasso Explained variance score for p20_1200LR : 0.19127843420168067
Lasso RMSE for p20_1200LIB : 0.12691810028222508
Lasso MAE for p20_1200LIB : 0.10278259018740207
Lasso R2 score for p20_1200LIB : 0.17237942652279048
Lasso Explained variance score for p20_1200LIB : 0.1912784332996199
Lasso RMSE for P20_1200DT : 0.12578319523779205
Lasso MAE for P20_1200DT : 0.10502021840128047
Lasso R2 score for P20_1200DT : 0.2570198087787331
Lasso Explained variance score for P20_1200DT : 0.2809267848037075
Lasso RMSE for P20_1200ridge : 0.1268824240058258
Lasso MAE for P20_1200ridge : 0.1089740706081154
Lasso R2 score for P20_1200ridge : 0.34995126469206916
Lasso Explained variance score for P20_1200ridge : 0.3790951198682261
Lasso RMSE for Predicted_p20_1200knn : 0.1305287730963825
Lasso MAE for Predicted_p20_1200knn : 0.10670374798112463
Lasso R2 score f

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: Futu

Lasso RMSE for p20_1200LR : 0.12280234628978667
Lasso MAE for p20_1200LR : 0.08978009243786889
Lasso R2 score for p20_1200LR : 0.15740304557749119
Lasso Explained variance score for p20_1200LR : 0.1694287234706331
Lasso RMSE for p20_1200LIB : 0.12280234633728689
Lasso MAE for p20_1200LIB : 0.08978009251322282
Lasso R2 score for p20_1200LIB : 0.1574030448300513
Lasso Explained variance score for p20_1200LIB : 0.1694287227490896
Lasso RMSE for P20_1200DT : 0.12838529441713928
Lasso MAE for P20_1200DT : 0.09633125342131268
Lasso R2 score for P20_1200DT : 0.21388562661357158
Lasso Explained variance score for P20_1200DT : 0.22527924829267298
Lasso RMSE for P20_1200ridge : 0.13764215819158712
Lasso MAE for P20_1200ridge : 0.10552846756667686
Lasso R2 score for P20_1200ridge : 0.2767457050371316
Lasso Explained variance score for P20_1200ridge : 0.28680818146311826
Lasso RMSE for Predicted_p20_1200knn : 0.13783587551930418
Lasso MAE for Predicted_p20_1200knn : 0.09720838423020943
Lasso R2 sc

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: Futu

...

Lasso RMSE for p20_1200LR : 0.08743580072766911
Lasso MAE for p20_1200LR : 0.0746853023735846
Lasso R2 score for p20_1200LR : -0.00032805007835468025
Lasso Explained variance score for p20_1200LR : 0.2722780489405242
Lasso RMSE for p20_1200LIB : 0.08743580070351445
Lasso MAE for p20_1200LIB : 0.0746853023887324
Lasso R2 score for p20_1200LIB : -0.0003280521765867128
Lasso Explained variance score for p20_1200LIB : 0.2722780482499946
Lasso RMSE for P20_1200DT : 0.09405433368009401
Lasso MAE for P20_1200DT : 0.08045691210569222
Lasso R2 score for P20_1200DT : 0.12587960697699307
Lasso Explained variance score for P20_1200DT : 0.32110913502369154
Lasso RMSE for P20_1200ridge : 0.10281899752298747
Lasso MAE for P20_1200ridge : 0.08960101638525977
Lasso R2 score for P20_1200ridge : 0.24049729451306
Lasso Explained variance score for P20_1200ridge : 0.3730325046646036
Lasso RMSE for Predicted_p20_1200knn : 0.09405355940655966
Lasso MAE for Predicted_p20_1200knn : 0.07984216734453976
Lasso R2

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: Futu

Lasso RMSE for p20_1200LR : 0.1551854495983331
Lasso MAE for p20_1200LR : 0.11817520659408332
Lasso R2 score for p20_1200LR : 0.1307841079182268
Lasso Explained variance score for p20_1200LR : 0.13508014842296234
Lasso RMSE for p20_1200LIB : 0.15518544964134307
Lasso MAE for p20_1200LIB : 0.11817520662399274
Lasso R2 score for p20_1200LIB : 0.13078410720348688
Lasso Explained variance score for p20_1200LIB : 0.13508014759433695
Lasso RMSE for P20_1200DT : 0.15590370581652233
Lasso MAE for P20_1200DT : 0.12186422765911929
Lasso R2 score for P20_1200DT : 0.2032161318583925
Lasso Explained variance score for P20_1200DT : 0.20645416097025993
Lasso RMSE for P20_1200ridge : 0.15950121891545127
Lasso MAE for P20_1200ridge : 0.12684217578430432
Lasso R2 score for P20_1200ridge : 0.2855880207357844
Lasso Explained variance score for P20_1200ridge : 0.2881450034516614
Lasso RMSE for Predicted_p20_1200knn : 0.1667759358851817
Lasso MAE for Predicted_p20_1200knn : 0.12670044870676334
Lasso R2 scor

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1918467545.py:57: Futu

Lasso(alpha=0.1, max_iter=100)

In [38]:
# Print the error DataFrame in a tabular format
print(error_df)

# Calculate the mean error
mean_lasso_rmse = np.mean(lasso_rmse_list)
mean_lasso_mae = np.mean(lasso_mae_list)
mean_lasso_r2 = np.mean(lasso_r2_list)
mean_lasso_explained_variance = np.mean(lasso_explained_variance_list)

print("Mean Lasso RMSE:", mean_lasso_rmse)
print("Mean Lasso MAE:", mean_lasso_mae)
print("Mean Lasso R2 Score:", mean_lasso_r2)
print("Mean Lasso Explained Variance:", mean_lasso_explained_variance)



          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.126918  0.102783  0.172379            0.191278
1             p20_1200LIB  0.126918  0.102783  0.172379            0.191278
2              P20_1200DT  0.125783  0.105020  0.257020            0.280927
3           P20_1200ridge  0.126882  0.108974  0.349951            0.379095
4   Predicted_p20_1200knn  0.130529  0.106704  0.318385            0.344646
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.224826  0.151383  0.062675            0.063316
66          P20_1200ridge  0.217529  0.154161  0.200091            0.200130
67  Predicted_p20_1200knn  0.224518  0.149348  0.157923            0.158467
68                   ANN2  0.216694  0.154650  0.216669            0.216684
69                     en  0.217931  0.153962  0.192203            0.192258

[70 rows x 5 columns]
Mean Lasso RMSE: 0.14225021644831273
Mean Lasso MAE: 0.1110410227

In [39]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
ridge_rmse_list = []
ridge_mae_list = []
ridge_r2_list = []
ridge_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Adjust the solver options as per your needs
    'max_iter': [1000, 2000, 5000]  # Include additional parameter options
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Ridge regression model
    ridge_model = Ridge()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ridge_model, ridge_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ridge_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_ridge_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ridge_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ridge_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ridge_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ridge_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ridge_rmse_list.append(ridge_rmse_val)
        ridge_mae_list.append(ridge_mae_val)
        ridge_r2_list.append(ridge_r2_val)
        ridge_explained_variance_list.append(ridge_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ridge_rmse_val,
            'MAE': ridge_mae_val,
            'R2 Score': ridge_r2_val,
            'Explained Variance': ridge_explained_variance_val
        }, ignore_index=True)

        print("Ridge RMSE for", target_variable, ":", ridge_rmse_val)
        print("Ridge MAE for", target_variable, ":", ridge_mae_val)
        print("Ridge R2 score for", target_variable, ":", ridge_r2_val)
        print("Ridge Explained variance score for", target_variable, ":", ridge_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)

# Train the model with the best parameters on the full training set
best_ridge_model.fit(X_train, y_train)



C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: Futu

Ridge RMSE for p20_1200LR : 0.03423273410507122
Ridge MAE for p20_1200LR : 0.0262342863769046
Ridge R2 score for p20_1200LR : 0.883937049184292
Ridge Explained variance score for p20_1200LR : 0.8886934447430002
Ridge RMSE for p20_1200LIB : 0.03423083693631093
Ridge MAE for p20_1200LIB : 0.026233511037012082
Ridge R2 score for p20_1200LIB : 0.8839499128632986
Ridge Explained variance score for p20_1200LIB : 0.888708456330167
Ridge RMSE for P20_1200DT : 0.03204487513494856
Ridge MAE for P20_1200DT : 0.02380570273301753
Ridge R2 score for P20_1200DT : 0.9150231037130744
Ridge Explained variance score for P20_1200DT : 0.9162721236524063
Ridge RMSE for P20_1200ridge : 0.03164570738826157
Ridge MAE for P20_1200ridge : 0.022611944482752458
Ridge R2 score for P20_1200ridge : 0.9349633916080925
Ridge Explained variance score for P20_1200ridge : 0.9349633987091074
Ridge RMSE for Predicted_p20_1200knn : 0.03914793475873716
Ridge MAE for Predicted_p20_1200knn : 0.028627204552584686
Ridge R2 score 

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: Futu

Ridge RMSE for p20_1200LR : 0.0361631987325625
Ridge MAE for p20_1200LR : 0.02647216988934997
Ridge R2 score for p20_1200LR : 0.9545570190899427
Ridge Explained variance score for p20_1200LR : 0.9552583121308619
Ridge RMSE for p20_1200LIB : 0.03616379174383798
Ridge MAE for p20_1200LIB : 0.02647233344582577
Ridge R2 score for p20_1200LIB : 0.95455552863222
Ridge Explained variance score for p20_1200LIB : 0.9552563436542344
Ridge RMSE for P20_1200DT : 0.03494157245142873
Ridge MAE for P20_1200DT : 0.025637121288531125
Ridge R2 score for P20_1200DT : 0.9630278890101817
Ridge Explained variance score for P20_1200DT : 0.9633947179410528
Ridge RMSE for P20_1200ridge : 0.03498591108460689
Ridge MAE for P20_1200ridge : 0.02489884101537978
Ridge R2 score for P20_1200ridge : 0.969523605316806
Ridge Explained variance score for P20_1200ridge : 0.9697215563939623
Ridge RMSE for Predicted_p20_1200knn : 0.04762091150459021
Ridge MAE for Predicted_p20_1200knn : 0.03330525652218353
Ridge R2 score for

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: Futu

...

Ridge RMSE for p20_1200LR : 0.0376501528143883
Ridge MAE for p20_1200LR : 0.027696307456995164
Ridge R2 score for p20_1200LR : 0.9390375372994222
Ridge Explained variance score for p20_1200LR : 0.9390582450183664
Ridge RMSE for p20_1200LIB : 0.037646960734320695
Ridge MAE for p20_1200LIB : 0.027695286073614715
Ridge R2 score for p20_1200LIB : 0.939047873847807
Ridge Explained variance score for p20_1200LIB : 0.9390687161607493
Ridge RMSE for P20_1200DT : 0.03388358266114444
Ridge MAE for P20_1200DT : 0.025339834024893426
Ridge R2 score for P20_1200DT : 0.9557501549100855
Ridge Explained variance score for P20_1200DT : 0.9558818602055072
Ridge RMSE for P20_1200ridge : 0.033995414575393006
Ridge MAE for P20_1200ridge : 0.025935935996169602
Ridge R2 score for P20_1200ridge : 0.9624241726073601
Ridge Explained variance score for P20_1200ridge : 0.9629498720335314
Ridge RMSE for Predicted_p20_1200knn : 0.0421815350850151
Ridge MAE for Predicted_p20_1200knn : 0.029694511638890676
Ridge R2 sc

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: Futu

Ridge RMSE for p20_1200LR : 0.03031033659021567
Ridge MAE for p20_1200LR : 0.022543940124988924
Ridge R2 score for p20_1200LR : 0.972562273952741
Ridge Explained variance score for p20_1200LR : 0.9753183641809728
Ridge RMSE for p20_1200LIB : 0.030306718749663637
Ridge MAE for p20_1200LIB : 0.022542318965335987
Ridge R2 score for p20_1200LIB : 0.9725688234540957
Ridge Explained variance score for p20_1200LIB : 0.9753242653511296
Ridge RMSE for P20_1200DT : 0.02828061577192063
Ridge MAE for P20_1200DT : 0.020753392835831632
Ridge R2 score for P20_1200DT : 0.9765000491596729
Ridge Explained variance score for P20_1200DT : 0.9774686267957738
Ridge RMSE for P20_1200ridge : 0.031518186020076416
Ridge MAE for P20_1200ridge : 0.02111575360148869
Ridge R2 score for P20_1200ridge : 0.9719885176909951
Ridge Explained variance score for P20_1200ridge : 0.9720500824607907
Ridge RMSE for Predicted_p20_1200knn : 0.0402224469541593
Ridge MAE for Predicted_p20_1200knn : 0.027133184321417262
Ridge R2 sc

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\4137435917.py:58: Futu

Ridge(alpha=0.1, max_iter=1000, solver='sag')

In [40]:
# Calculate the mean error for Ridge regression
mean_ridge_rmse = np.mean(ridge_rmse_list)
mean_ridge_mae = np.mean(ridge_mae_list)
mean_ridge_r2 = np.mean(ridge_r2_list)
mean_ridge_explained_variance = np.mean(ridge_explained_variance_list)

print("Mean Ridge RMSE:", mean_ridge_rmse)
print("Mean Ridge MAE:", mean_ridge_mae)
print("Mean Ridge R2 Score:", mean_ridge_r2)
print("Mean Ridge Explained Variance:", mean_ridge_explained_variance)


Mean Ridge RMSE: 0.0359662760689557
Mean Ridge MAE: 0.025897283958297504
Mean Ridge R2 Score: 0.9403824919781042
Mean Ridge Explained Variance: 0.9422224389505376


In [41]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
knn_rmse_list = []
knn_mae_list = []
knn_r2_list = []
knn_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
knn_param_grid = {
    'n_neighbors': [3, 5, 7],  # Adjust the number of neighbors as per your needs
    'weights': ['uniform', 'distance'],  # Adjust the weight options as per your needs
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Adjust the algorithm options as per your needs
    'leaf_size': [10, 30, 50],  # Adjust the leaf size options as per your needs
    'p': [1, 2],  # Adjust the distance metric options as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the KNN regression model
    knn_model = KNeighborsRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(knn_model, knn_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_knn_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_knn_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        knn_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        knn_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        knn_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        knn_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        knn_rmse_list.append(knn_rmse_val)
        knn_mae_list.append(knn_mae_val)
        knn_r2_list.append(knn_r2_val)
        knn_explained_variance_list.append(knn_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': knn_rmse_val,
            'MAE': knn_mae_val,
            'R2 Score': knn_r2_val,
            'Explained Variance': knn_explained_variance_val
        }, ignore_index=True)

        print("KNN RMSE for", target_variable, ":", knn_rmse_val)
        print("KNN MAE for", target_variable, ":", knn_mae_val)
        print("KNN R2 score for", target_variable, ":", knn_r2_val)
        print("KNN Explained variance score for", target_variable, ":", knn_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)




C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

KNN RMSE for p20_1200LR : 0.09073167296427945
KNN MAE for p20_1200LR : 0.03811495046132584
KNN R2 score for p20_1200LR : 0.8406240324131603
KNN Explained variance score for p20_1200LR : 0.8527734691276175
KNN RMSE for p20_1200LIB : 0.09073167300193205
KNN MAE for p20_1200LIB : 0.038114950614563325
KNN R2 score for p20_1200LIB : 0.8406240321297374
KNN Explained variance score for p20_1200LIB : 0.8527734689537942
KNN RMSE for P20_1200DT : 0.08242509390004993
KNN MAE for P20_1200DT : 0.03593311765278835
KNN R2 score for P20_1200DT : 0.8675316566985573
KNN Explained variance score for P20_1200DT : 0.878082742604156
KNN RMSE for P20_1200ridge : 0.07071212689026297
KNN MAE for P20_1200ridge : 0.03333991589755165
KNN R2 score for P20_1200ridge : 0.9036512458273023
KNN Explained variance score for P20_1200ridge : 0.91205945076389
KNN RMSE for Predicted_p20_1200knn : 0.07034355220940003
KNN MAE for Predicted_p20_1200knn : 0.03213381274280409
KNN R2 score for Predicted_p20_1200knn : 0.9094569079

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

KNN RMSE for p20_1200LR : 0.027177942534653186
KNN MAE for p20_1200LR : 0.021224971849140777
KNN R2 score for p20_1200LR : 0.9345525240705788
KNN Explained variance score for p20_1200LR : 0.9349802386669315
KNN RMSE for p20_1200LIB : 0.027177942407100295
KNN MAE for p20_1200LIB : 0.021224971757524972
KNN R2 score for p20_1200LIB : 0.9345525245736866
KNN Explained variance score for p20_1200LIB : 0.9349802391977328
KNN RMSE for P20_1200DT : 0.026839700880308664
KNN MAE for P20_1200DT : 0.021928835544016815
KNN R2 score for P20_1200DT : 0.9433648577538056
KNN Explained variance score for P20_1200DT : 0.9446145792704045
KNN RMSE for P20_1200ridge : 0.029634619541098556
KNN MAE for P20_1200ridge : 0.024109901532851006
KNN R2 score for P20_1200ridge : 0.9423990013495616
KNN Explained variance score for P20_1200ridge : 0.9454596417900527
KNN RMSE for Predicted_p20_1200knn : 0.0279897726335893
KNN MAE for Predicted_p20_1200knn : 0.02234970068136773
KNN R2 score for Predicted_p20_1200knn : 0.9

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

...

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

KNN RMSE for p20_1200LR : 0.062156895948348286
KNN MAE for p20_1200LR : 0.03305492810785384
KNN R2 score for p20_1200LR : 0.9033260497488216
KNN Explained variance score for p20_1200LR : 0.9090889841820983
KNN RMSE for p20_1200LIB : 0.06215689598411091
KNN MAE for p20_1200LIB : 0.033054928021056126
KNN R2 score for p20_1200LIB : 0.9033260495388624
KNN Explained variance score for p20_1200LIB : 0.9090889839456315
KNN RMSE for P20_1200DT : 0.06303208410931901
KNN MAE for P20_1200DT : 0.03397335740899359
KNN R2 score for P20_1200DT : 0.9090720568626993
KNN Explained variance score for P20_1200DT : 0.9141693669861779
KNN RMSE for P20_1200ridge : 0.06635231072121588
KNN MAE for P20_1200ridge : 0.03669050215363902
KNN R2 score for P20_1200ridge : 0.9130207511026829
KNN Explained variance score for P20_1200ridge : 0.9178646938475105
KNN RMSE for Predicted_p20_1200knn : 0.07661217265166063
KNN MAE for Predicted_p20_1200knn : 0.03847512385604862
KNN R2 score for Predicted_p20_1200knn : 0.893215

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

KNN RMSE for p20_1200LR : 0.04328874041085126
KNN MAE for p20_1200LR : 0.01945987450500452
KNN R2 score for p20_1200LR : 0.8615720834228238
KNN Explained variance score for p20_1200LR : 0.8727036493672432
KNN RMSE for p20_1200LIB : 0.043288740352993134
KNN MAE for p20_1200LIB : 0.019459874481896452
KNN R2 score for p20_1200LIB : 0.8615720834822085
KNN Explained variance score for p20_1200LIB : 0.8727036495269435
KNN RMSE for P20_1200DT : 0.04125579144131807
KNN MAE for P20_1200DT : 0.019562987957255282
KNN R2 score for P20_1200DT : 0.8908211040220482
KNN Explained variance score for P20_1200DT : 0.9000134747388251
KNN RMSE for P20_1200ridge : 0.03918273118284208
KNN MAE for P20_1200ridge : 0.019790241079355852
KNN R2 score for P20_1200ridge : 0.9187887328547564
KNN Explained variance score for P20_1200ridge : 0.9258717903775006
KNN RMSE for Predicted_p20_1200knn : 0.05150892884598456
KNN MAE for Predicted_p20_1200knn : 0.02368851823578184
KNN R2 score for Predicted_p20_1200knn : 0.8546

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\425568073.py:60: FutureWar

In [42]:
# Calculate the mean error for KNN Regression
knn_mean_rmse = np.mean(knn_rmse_list)
knn_mean_mae = np.mean(knn_mae_list)
knn_mean_r2 = np.mean(knn_r2_list)
knn_mean_explained_variance = np.mean(knn_explained_variance_list)

print("\nMean KNN RMSE:", knn_mean_rmse)
print("Mean KNN MAE:", knn_mean_mae)
print("Mean KNN R2 Score:", knn_mean_r2)
print("Mean KNN Explained Variance:", knn_mean_explained_variance)


Mean KNN RMSE: 0.0414273225723223
Mean KNN MAE: 0.023878365597097224
Mean KNN R2 Score: 0.9236084087934893
Mean KNN Explained Variance: 0.9273886296889949


In [43]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
ann_rmse_list = []
ann_mae_list = []
ann_r2_list = []
ann_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
ann_param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 50, 50)],  # Adjust the hidden layer sizes as per your needs
    'activation': ['relu', 'tanh'],  # Adjust the activation function options as per your needs
    'solver': ['adam', 'sgd'],  # Adjust the solver options as per your needs
    'learning_rate': ['constant', 'adaptive'],  # Adjust the learning rate options as per your needs
    'alpha': [0.0001],  # Adjust the regularization parameter as per your needs
    'batch_size': ['auto'],  # Adjust the batch size as per your needs
    'learning_rate_init': [0.001],  # Adjust the initial learning rate as per your needs
    'power_t': [0.5],  # Adjust the power parameter as per your needs
    'max_iter': [200],  # Adjust the maximum number of iterations as per your needs
    'shuffle': [True],  # Adjust the shuffle parameter as per your needs
    'random_state': [None],  # Adjust the random state as per your needs
    'tol': [0.0001],  # Adjust the tolerance as per your needs
    'verbose': [False],  # Adjust the verbosity level as per your needs
    'warm_start': [False],  # Adjust the warm start parameter as per your needs
    'momentum': [0.9],  # Adjust the momentum parameter as per your needs
    'nesterovs_momentum': [True],  # Adjust the Nesterov's momentum parameter as per your needs
    'early_stopping': [False],  # Adjust the early stopping parameter as per your needs
    'validation_fraction': [0.1],  # Adjust the validation fraction as per your needs
    'beta_1': [0.9],  # Adjust the beta 1 parameter as per your needs
    'beta_2': [0.999],  # Adjust the beta 2 parameter as per your needs
    'epsilon': [1e-08],  # Adjust the epsilon parameter as per your needs
    'n_iter_no_change': [10],  # Adjust the number of iterations with no improvement as per your needs
    'max_fun': [15000]  # Adjust the maximum number of function evaluations as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the ANN regression model
    ann_model = MLPRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ann_model, ann_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ann_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_ann_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_ann_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ann_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ann_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ann_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ann_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ann_rmse_list.append(ann_rmse_val)
        ann_mae_list.append(ann_mae_val)
        ann_r2_list.append(ann_r2_val)
        ann_explained_variance_list.append(ann_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ann_rmse_val,
            'MAE': ann_mae_val,
            'R2 Score': ann_r2_val,
            'Explained Variance': ann_explained_variance_val
        }, ignore_index=True)

        print("ANN RMSE for", target_variable, ":", ann_rmse_val)
        print("ANN MAE for", target_variable, ":", ann_mae_val)
        print("ANN R2 score for", target_variable, ":", ann_r2_val)
        print("ANN Explained variance score for", target_variable, ":", ann_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)


C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

ANN RMSE for p20_1200LR : 0.03712576248049109
ANN MAE for p20_1200LR : 0.02621090039136992
ANN R2 score for p20_1200LR : 0.7267983998953986
ANN Explained variance score for p20_1200LR : 0.7289747009125046
ANN RMSE for p20_1200LIB : 0.0765837634733992
ANN MAE for p20_1200LIB : 0.05099835295342425
ANN R2 score for p20_1200LIB : -0.1625345911095788
ANN Explained variance score for p20_1200LIB : 0.30386349439622573
ANN RMSE for P20_1200DT : 0.07141924196424236
ANN MAE for P20_1200DT : 0.06593893026116332
ANN R2 score for P20_1200DT : 0.26549289528115594
ANN Explained variance score for P20_1200DT : 0.6246526448359089
ANN RMSE for P20_1200ridge : 0.02752436636438636
ANN MAE for P20_1200ridge : 0.021360527338377764
ANN R2 score for P20_1200ridge : 0.9237883868203672
ANN Explained variance score for P20_1200ridge : 0.9290372286778905
ANN RMSE for Predicted_p20_1200knn : 0.05673277110752468
ANN MAE for Predicted_p20_1200knn : 0.040184174775966586
ANN R2 score for Predicted_p20_1200knn : 0.5642

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

ANN RMSE for p20_1200LR : 0.042692358815665694
ANN MAE for p20_1200LR : 0.03299607948418122
ANN R2 score for p20_1200LR : 0.9140435980841932
ANN Explained variance score for p20_1200LR : 0.9140830747437302
ANN RMSE for p20_1200LIB : 0.05392266108755348
ANN MAE for p20_1200LIB : 0.0375609720656335
ANN R2 score for p20_1200LIB : 0.8628737625060734
ANN Explained variance score for p20_1200LIB : 0.8829538782841025
ANN RMSE for P20_1200DT : 0.03407229613221529
ANN MAE for P20_1200DT : 0.025175863116939857
ANN R2 score for P20_1200DT : 0.9541927783248991
ANN Explained variance score for P20_1200DT : 0.9542250405803703
ANN RMSE for P20_1200ridge : 0.03802349456584431
ANN MAE for P20_1200ridge : 0.029492571190291215
ANN R2 score for P20_1200ridge : 0.9549949016237104
ANN Explained variance score for P20_1200ridge : 0.9603686924292679
ANN RMSE for Predicted_p20_1200knn : 0.03202062578395749
ANN MAE for Predicted_p20_1200knn : 0.02291552394449621
ANN R2 score for Predicted_p20_1200knn : 0.966350

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

...

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

ANN RMSE for p20_1200LR : 0.04959776156779522
ANN MAE for p20_1200LR : 0.03683201520798363
ANN R2 score for p20_1200LR : 0.9151675493602356
ANN Explained variance score for p20_1200LR : 0.9157816321876664
ANN RMSE for p20_1200LIB : 0.05160507513355071
ANN MAE for p20_1200LIB : 0.03722104801889893
ANN R2 score for p20_1200LIB : 0.9081619422978026
ANN Explained variance score for p20_1200LIB : 0.9084953194208182
ANN RMSE for P20_1200DT : 0.03977011568182069
ANN MAE for P20_1200DT : 0.029854400199683828
ANN R2 score for P20_1200DT : 0.9525310801621748
ANN Explained variance score for P20_1200DT : 0.9578549605505597
ANN RMSE for P20_1200ridge : 0.05476636480277594
ANN MAE for P20_1200ridge : 0.036358042648967435
ANN R2 score for P20_1200ridge : 0.9263148678608533
ANN Explained variance score for P20_1200ridge : 0.9419103700728337
ANN RMSE for Predicted_p20_1200knn : 0.08218974390070881
ANN MAE for Predicted_p20_1200knn : 0.06720646058720464
ANN R2 score for Predicted_p20_1200knn : 0.838676

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

ANN RMSE for p20_1200LR : 0.035409047808889725
ANN MAE for p20_1200LR : 0.028038321440770352
ANN R2 score for p20_1200LR : 0.950933339041423
ANN Explained variance score for p20_1200LR : 0.9644385446742731
ANN RMSE for p20_1200LIB : 0.04053994677534562
ANN MAE for p20_1200LIB : 0.03072979578044821
ANN R2 score for p20_1200LIB : 0.9356832106246924
ANN Explained variance score for p20_1200LIB : 0.9506110086973721
ANN RMSE for P20_1200DT : 0.03657892415437284
ANN MAE for P20_1200DT : 0.027197608810699877
ANN R2 score for P20_1200DT : 0.9532357437749539
ANN Explained variance score for P20_1200DT : 0.9541258972280655
ANN RMSE for P20_1200ridge : 0.03814402425688864
ANN MAE for P20_1200ridge : 0.030936489609806214
ANN R2 score for P20_1200ridge : 0.9570400483546186
ANN Explained variance score for P20_1200ridge : 0.9572279114475452
ANN RMSE for Predicted_p20_1200knn : 0.04852907627059974
ANN MAE for Predicted_p20_1200knn : 0.0348293558999428
ANN R2 score for Predicted_p20_1200knn : 0.935460

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3887435190.py:80: Futu

In [44]:
# Calculate the mean error for ANN Regression
ann_mean_rmse = np.mean(ann_rmse_list)
ann_mean_mae = np.mean(ann_mae_list)
ann_mean_r2 = np.mean(ann_r2_list)
ann_mean_explained_variance = np.mean(ann_explained_variance_list)

print("\nMean ANN RMSE:", ann_mean_rmse)
print("Mean ANN MAE:", ann_mean_mae)
print("Mean ANN R2 Score:", ann_mean_r2)
print("Mean ANN Explained Variance:", ann_mean_explained_variance)





Mean ANN RMSE: 0.04759955832396334
Mean ANN MAE: 0.03491080841601468
Mean ANN R2 Score: 0.8734877243318563
Mean ANN Explained Variance: 0.8999294288372874


In [45]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import ElasticNet
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
en_rmse_list = []
en_mae_list = []
en_r2_list = []
en_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
en_param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'l1_ratio': [0.2, 0.5, 0.8],  # Adjust the values of l1_ratio as per your needs
    'fit_intercept': [True, False],  # Adjust the fit_intercept option as per your needs
    'precompute': [True, False],  # Adjust the precompute option as per your needs
    'max_iter': [1000],  # Adjust the maximum number of iterations as per your needs
    'tol': [0.0001],  # Adjust the tolerance as per your needs
    'warm_start': [True, False],  # Adjust the warm_start option as per your needs
    'positive': [False],  # Adjust the positive option as per your needs
    'random_state': [None],  # Adjust the random_state as per your needs
    'selection': ['cyclic', 'random']  # Adjust the selection option as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Elastic Net regression model
    en_model = ElasticNet()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(en_model, en_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_en_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_en_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_en_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        en_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        en_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        en_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        en_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        en_rmse_list.append(en_rmse_val)
        en_mae_list.append(en_mae_val)
        en_r2_list.append(en_r2_val)
        en_explained_variance_list.append(en_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': en_rmse_val,
            'MAE': en_mae_val,
            'R2 Score': en_r2_val,
            'Explained Variance': en_explained_variance_val
        }, ignore_index=True)

        print("Elastic Net RMSE for", target_variable, ":", en_rmse_val)
        print("Elastic Net MAE for", target_variable, ":", en_mae_val)
        print("Elastic Net R2 score for", target_variable, ":", en_r2_val)
        print("Elastic Net Explained variance score for", target_variable, ":", en_explained_variance_val)







C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

Elastic Net RMSE for p20_1200LR : 0.07290132415164775
Elastic Net MAE for p20_1200LR : 0.05091603854912216
Elastic Net R2 score for p20_1200LR : 0.697684988962991
Elastic Net Explained variance score for p20_1200LR : 0.7053527441999361
Elastic Net RMSE for p20_1200LIB : 0.07290130716359129
Elastic Net MAE for p20_1200LIB : 0.05091603599908711
Elastic Net R2 score for p20_1200LIB : 0.6976851294673831
Elastic Net Explained variance score for p20_1200LIB : 0.7053529139630483
Elastic Net RMSE for P20_1200DT : 0.07535891841593778
Elastic Net MAE for P20_1200DT : 0.05378649543939262
Elastic Net R2 score for P20_1200DT : 0.7304208265723582
Elastic Net Explained variance score for P20_1200DT : 0.7342857375449265
Elastic Net RMSE for P20_1200ridge : 0.08102370718925399
Elastic Net MAE for P20_1200ridge : 0.05882288209324797
Elastic Net R2 score for P20_1200ridge : 0.7560079103571469
Elastic Net Explained variance score for P20_1200ridge : 0.7573971434709345
Elastic Net RMSE for Predicted_p20_12

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

Elastic Net RMSE for p20_1200LR : 0.05055293349089969
Elastic Net MAE for p20_1200LR : 0.03804701566110705
Elastic Net R2 score for p20_1200LR : 0.7372416940603913
Elastic Net Explained variance score for p20_1200LR : 0.7499228902094239
Elastic Net RMSE for p20_1200LIB : 0.05055293362142977
Elastic Net MAE for p20_1200LIB : 0.03804701577760481
Elastic Net R2 score for p20_1200LIB : 0.737241692760456
Elastic Net Explained variance score for p20_1200LIB : 0.7499228889030782
Elastic Net RMSE for P20_1200DT : 0.05411865843683715
Elastic Net MAE for P20_1200DT : 0.04025466133766889
Elastic Net R2 score for P20_1200DT : 0.7628806175717269
Elastic Net Explained variance score for P20_1200DT : 0.7711189585497458
Elastic Net RMSE for P20_1200ridge : 0.060009998265534124
Elastic Net MAE for P20_1200ridge : 0.044816020430438155
Elastic Net R2 score for P20_1200ridge : 0.7815391021007251
Elastic Net Explained variance score for P20_1200ridge : 0.7869834097219213
Elastic Net RMSE for Predicted_p20_

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

...

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

Elastic Net RMSE for p20_1200LR : 0.1032780200730768
Elastic Net MAE for p20_1200LR : 0.06675002547884258
Elastic Net R2 score for p20_1200LR : 0.6291292072116792
Elastic Net Explained variance score for p20_1200LR : 0.6641956831012741
Elastic Net RMSE for p20_1200LIB : 0.10327802005677854
Elastic Net MAE for p20_1200LIB : 0.06675002546265531
Elastic Net R2 score for p20_1200LIB : 0.6291292068673493
Elastic Net Explained variance score for p20_1200LIB : 0.6641956827750419
Elastic Net RMSE for P20_1200DT : 0.10348081177487718
Elastic Net MAE for P20_1200DT : 0.06878562447340751
Elastic Net R2 score for P20_1200DT : 0.6631477207129637
Elastic Net Explained variance score for P20_1200DT : 0.6978022467415472
Elastic Net RMSE for P20_1200ridge : 0.10717785080801008
Elastic Net MAE for P20_1200ridge : 0.07222399737726791
Elastic Net R2 score for P20_1200ridge : 0.693322390099157
Elastic Net Explained variance score for P20_1200ridge : 0.7272868426448906
Elastic Net RMSE for Predicted_p20_120

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

Elastic Net RMSE for p20_1200LR : 0.1293062125863293
Elastic Net MAE for p20_1200LR : 0.08521011896498915
Elastic Net R2 score for p20_1200LR : 0.6558106512616573
Elastic Net Explained variance score for p20_1200LR : 0.6980308293564461
Elastic Net RMSE for p20_1200LIB : 0.12930621261479366
Elastic Net MAE for p20_1200LIB : 0.0852101190542179
Elastic Net R2 score for p20_1200LIB : 0.6558106509150492
Elastic Net Explained variance score for p20_1200LIB : 0.698030828922885
Elastic Net RMSE for P20_1200DT : 0.12295964977296833
Elastic Net MAE for P20_1200DT : 0.08547973596690893
Elastic Net R2 score for P20_1200DT : 0.7093789941615161
Elastic Net Explained variance score for P20_1200DT : 0.7395397968580868
Elastic Net RMSE for P20_1200ridge : 0.11819727232705982
Elastic Net MAE for P20_1200ridge : 0.08744699263513979
Elastic Net R2 score for P20_1200ridge : 0.761335916014155
Elastic Net Explained variance score for P20_1200ridge : 0.7806543034544304
Elastic Net RMSE for Predicted_p20_1200k

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1202926302.py:67: Futu

In [46]:
# Calculate the mean errors for each target variable
en_rmse_mean = np.mean(en_rmse_list)
en_mae_mean = np.mean(en_mae_list)
en_r2_mean = np.mean(en_r2_list)
en_explained_variance_mean = np.mean(en_explained_variance_list)

print("\nMean Elastic Net RMSE:", en_rmse_mean)
print("Mean Elastic Net MAE:", en_mae_mean)
print("Mean Elastic Net R2 score:", en_r2_mean)
print("Mean Elastic Net Explained variance score:", en_explained_variance_mean)

# Print the error DataFrame in a tabular format
print("\nError DataFrame:")
print(error_df)




Mean Elastic Net RMSE: 0.08282389812018091
Mean Elastic Net MAE: 0.06083387210981923
Mean Elastic Net R2 score: 0.7142751198631088
Mean Elastic Net Explained variance score: 0.744179593192153

Error DataFrame:
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.072901  0.050916  0.697685            0.705353
1             p20_1200LIB  0.072901  0.050916  0.697685            0.705353
2              P20_1200DT  0.075359  0.053786  0.730421            0.734286
3           P20_1200ridge  0.081024  0.058823  0.756008            0.757397
4   Predicted_p20_1200knn  0.085729  0.057511  0.714580            0.717600
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.049541  0.037036  0.787622            0.810396
66          P20_1200ridge  0.055703  0.040799  0.799567            0.811741
67  Predicted_p20_1200knn  0.055810  0.041373  0.765663            0.790990
68                   ANN2  0.

In [47]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import pandas as pd

# Define lists to store the errors for each target variable
dt_rmse_list = []
dt_mae_list = []
dt_r2_list = []
dt_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
dt_param_grid = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Adjust the criterion options as per your needs
    'max_depth': [None, 5, 10],  # Adjust the maximum depth options as per your needs
    'min_samples_split': [2, 5, 10],  # Adjust the minimum samples split options as per your needs
    'min_samples_leaf': [1, 2, 4],  # Adjust the minimum samples leaf options as per your needs
    'random_state': [42]  # Adjust the random state as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Decision Tree regression model
    dt_model = DecisionTreeRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(dt_model, dt_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_dt_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_dt_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_dt_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        dt_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        dt_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        dt_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        dt_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        dt_rmse_list.append(dt_rmse_val)
        dt_mae_list.append(dt_mae_val)
        dt_r2_list.append(dt_r2_val)
        dt_explained_variance_list.append(dt_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': dt_rmse_val,
            'MAE': dt_mae_val,
            'R2 Score': dt_r2_val,
            'Explained Variance': dt_explained_variance_val
        }, ignore_index=True)

        print("Decision Tree RMSE for", target_variable, ":", dt_rmse_val)
        print("Decision Tree MAE for", target_variable, ":", dt_mae_val)
        print("Decision Tree R2 score for", target_variable, ":", dt_r2_val)
        print("Decision Tree Explained variance score for", target_variable, ":", dt_explained_variance_val)



C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

Decision Tree RMSE for p20_1200LR : 0.07549960832218361
Decision Tree MAE for p20_1200LR : 0.04159810970241047
Decision Tree R2 score for p20_1200LR : 0.6386528499106198
Decision Tree Explained variance score for p20_1200LR : 0.6491399314514317
Decision Tree RMSE for p20_1200LIB : 0.07549960829482115
Decision Tree MAE for p20_1200LIB : 0.04159810977134264
Decision Tree R2 score for p20_1200LIB : 0.638652849757656
Decision Tree Explained variance score for p20_1200LIB : 0.6491399314390303
Decision Tree RMSE for P20_1200DT : 0.07678928473199563
Decision Tree MAE for P20_1200DT : 0.04594198559642402
Decision Tree R2 score for P20_1200DT : 0.6824058922590157
Decision Tree Explained variance score for P20_1200DT : 0.6892149088000494
Decision Tree RMSE for P20_1200ridge : 0.08142360297140708
Decision Tree MAE for P20_1200ridge : 0.05236442423582237
Decision Tree R2 score for P20_1200ridge : 0.7142121090329747
Decision Tree Explained variance score for P20_1200ridge : 0.7178803937512547
Decis

C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

Decision Tree RMSE for p20_1200LR : 0.038760487973391954
Decision Tree MAE for p20_1200LR : 0.022653905407897742
Decision Tree R2 score for p20_1200LR : 0.9238899379193423
Decision Tree Explained variance score for p20_1200LR : 0.9252954651944527
Decision Tree RMSE for p20_1200LIB : 0.03876048801842051
Decision Tree MAE for p20_1200LIB : 0.022653905424482854
Decision Tree R2 score for p20_1200LIB : 0.9238899376118292
Decision Tree Explained variance score for p20_1200LIB : 0.9252954647990814
Decision Tree RMSE for P20_1200DT : 0.041575760465440266
Decision Tree MAE for P20_1200DT : 0.02411411051867258
Decision Tree R2 score for P20_1200DT : 0.9250595432056122
Decision Tree Explained variance score for P20_1200DT : 0.9271844132546825
Decision Tree RMSE for P20_1200ridge : 0.046617880004137004
Decision Tree MAE for P20_1200ridge : 0.026842791010921307
Decision Tree R2 score for P20_1200ridge : 0.9240723072601618
Decision Tree Explained variance score for P20_1200ridge : 0.926963615038778

C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

...

C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

Decision Tree RMSE for p20_1200LR : 0.036346973013707325
Decision Tree MAE for p20_1200LR : 0.025436802056232343
Decision Tree R2 score for p20_1200LR : 0.9637125540026082
Decision Tree Explained variance score for p20_1200LR : 0.9673500891625204
Decision Tree RMSE for p20_1200LIB : 0.0363469730625192
Decision Tree MAE for p20_1200LIB : 0.02543680207630378
Decision Tree R2 score for p20_1200LIB : 0.9637125538662411
Decision Tree Explained variance score for p20_1200LIB : 0.9673500889655381
Decision Tree RMSE for P20_1200DT : 0.042215323291677105
Decision Tree MAE for P20_1200DT : 0.0295545224235789
Decision Tree R2 score for P20_1200DT : 0.9562255318470988
Decision Tree Explained variance score for P20_1200DT : 0.9600496499023334
Decision Tree RMSE for P20_1200ridge : 0.05294836391039275
Decision Tree MAE for P20_1200ridge : 0.03531293241189808
Decision Tree R2 score for P20_1200ridge : 0.9422104968501854
Decision Tree Explained variance score for P20_1200ridge : 0.9460635973578692
Dec

C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

Decision Tree RMSE for p20_1200LR : 0.024577983217689302
Decision Tree MAE for p20_1200LR : 0.01881615878059376
Decision Tree R2 score for p20_1200LR : 0.9566692755013738
Decision Tree Explained variance score for p20_1200LR : 0.9585299778243698
Decision Tree RMSE for p20_1200LIB : 0.024577983101360112
Decision Tree MAE for p20_1200LIB : 0.018816158648429355
Decision Tree R2 score for p20_1200LIB : 0.9566692758005322
Decision Tree Explained variance score for p20_1200LIB : 0.9585299780631791
Decision Tree RMSE for P20_1200DT : 0.032829140569905976
Decision Tree MAE for P20_1200DT : 0.024725553885711734
Decision Tree R2 score for P20_1200DT : 0.9300112170713581
Decision Tree Explained variance score for P20_1200DT : 0.9343832924924449
Decision Tree RMSE for P20_1200ridge : 0.04672593811628748
Decision Tree MAE for P20_1200ridge : 0.03269807623400287
Decision Tree R2 score for P20_1200ridge : 0.8793606218004355
Decision Tree Explained variance score for P20_1200ridge : 0.888101288795045


C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
540 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 1247, in fit
    super().fit(
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 177, in fit
    self._validate_params()
  File "C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py", li

In [48]:
# Calculate the mean errors for each target variable
dt_rmse_mean = np.mean(dt_rmse_list)
dt_mae_mean = np.mean(dt_mae_list)
dt_r2_mean = np.mean(dt_r2_list)
dt_explained_variance_mean = np.mean(dt_explained_variance_list)

print("\nMean Decision Tree RMSE:", dt_rmse_mean)
print("Mean Decision Tree MAE:", dt_mae_mean)
print("Mean Decision Tree R2 score:", dt_r2_mean)
print("Mean Decision Tree Explained variance score:", dt_explained_variance_mean)

# Print the error DataFrame in a tabular format
print("\nError DataFrame:")
print(error_df)



Mean Decision Tree RMSE: 0.058836235044083605
Mean Decision Tree MAE: 0.03643475968124564
Mean Decision Tree R2 score: 0.83754677318925
Mean Decision Tree Explained variance score: 0.8494526367726405

Error DataFrame:
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.075500  0.041598  0.638653            0.649140
1             p20_1200LIB  0.075500  0.041598  0.638653            0.649140
2              P20_1200DT  0.076789  0.045942  0.682406            0.689215
3           P20_1200ridge  0.081424  0.052364  0.714212            0.717880
4   Predicted_p20_1200knn  0.084292  0.046637  0.671923            0.680762
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.051308  0.033126  0.906088            0.918115
66          P20_1200ridge  0.061554  0.038783  0.888553            0.900053
67  Predicted_p20_1200knn  0.059091  0.037301  0.897513            0.907539
68                   

In [49]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import RANSACRegressor
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
ransac_rmse_list = []
ransac_mae_list = []
ransac_r2_list = []
ransac_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
param_grid = {
    'base_estimator': [None],  # Adjust the base estimator as per your needs
    'min_samples': [0.1, 0.5, 1.0],  # Adjust the minimum number of samples as per your needs
    'residual_threshold': [0.1, 1.0, 5.0],  # Adjust the residual threshold as per your needs
    'max_trials': [100, 500, 1000],  # Adjust the maximum number of trials as per your needs
    'max_skips': [np.inf],  # Adjust the maximum number of skips as per your needs
    'stop_probability': [0.99],  # Adjust the stop probability as per your needs
    'stop_score': [np.inf],  # Adjust the stop score as per your needs
    'loss': ['absolute_error', 'squared_error'],  # Adjust the loss function as per your needs
    'random_state': [42]  # Adjust the random state as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the RANSAC regression model
    ransac_model = RANSACRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ransac_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ransac_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_ransac_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ransac_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ransac_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ransac_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ransac_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ransac_rmse_list.append(ransac_rmse_val)
        ransac_mae_list.append(ransac_mae_val)
        ransac_r2_list.append(ransac_r2_val)
        ransac_explained_variance_list.append(ransac_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ransac_rmse_val,
            'MAE': ransac_mae_val,
            'R2 Score': ransac_r2_val,
            'Explained Variance': ransac_explained_variance_val
        }, ignore_index=True)

        print("RANSAC RMSE for", target_variable, ":", ransac_rmse_val)
        print("RANSAC MAE for", target_variable, ":", ransac_mae_val)
        print("RANSAC R2 score for", target_variable, ":", ransac_r2_val)
        print("RANSAC Explained variance score for", target_variable, ":", ransac_explained_variance_val)






C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

...

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
C:\Pr

RANSAC RMSE for p20_1200LR : 0.035630658037189
RANSAC MAE for p20_1200LR : 0.02692547122270912
RANSAC R2 score for p20_1200LR : 0.9454219482970577
RANSAC Explained variance score for p20_1200LR : 0.945422053096525
RANSAC RMSE for p20_1200LIB : 0.035630658050472795
RANSAC MAE for p20_1200LIB : 0.026925471230871245
RANSAC R2 score for p20_1200LIB : 0.945421948255935
RANSAC Explained variance score for p20_1200LIB : 0.9454220530559363
RANSAC RMSE for P20_1200DT : 0.03622939825419692
RANSAC MAE for P20_1200DT : 0.0280358521111469
RANSAC R2 score for P20_1200DT : 0.9494501602846702
RANSAC Explained variance score for P20_1200DT : 0.9494649494763996
RANSAC RMSE for P20_1200ridge : 0.0398750044019203
RANSAC MAE for P20_1200ridge : 0.03149837499305069
RANSAC R2 score for P20_1200ridge : 0.9481398079346637
RANSAC Explained variance score for P20_1200ridge : 0.948236511014013
RANSAC RMSE for Predicted_p20_1200knn : 0.0416478482745212
RANSAC MAE for Predicted_p20_1200knn : 0.031044613388223494
RA

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3399254261.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


In [50]:
# Calculate the mean errors for each target variable
mean_rmse = np.mean(ransac_rmse_list)
mean_mae = np.mean(ransac_mae_list)
mean_r2 = np.mean(ransac_r2_list)
mean_explained_variance = np.mean(ransac_explained_variance_list)

print("\nMean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 score:", mean_r2)
print("Mean Explained variance score:", mean_explained_variance)

# Print the error DataFrame in a tabular format
print(error_df)


Mean RMSE: 0.03606652925726061
Mean MAE: 0.02590879930109747
Mean R2 score: 0.935217302046629
Mean Explained variance score: 0.9406502223271443
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.027454  0.023341  0.943831            0.946662
1             p20_1200LIB  0.027454  0.023341  0.943831            0.946662
2              P20_1200DT  0.032002  0.026097  0.927470            0.931617
3           P20_1200ridge  0.039541  0.030435  0.899006            0.905464
4   Predicted_p20_1200knn  0.034506  0.028648  0.925504            0.930438
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.036229  0.028036  0.949450            0.949465
66          P20_1200ridge  0.039875  0.031498  0.948140            0.948237
67  Predicted_p20_1200knn  0.041648  0.031045  0.943845            0.944222
68                   ANN2  0.040378  0.032013  0.947970            0.948056
69                 

In [51]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
gpr_rmse_list = []
gpr_mae_list = []
gpr_r2_list = []
gpr_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'kernel': [RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
               RBF(length_scale=0.5, length_scale_bounds=(1e-1, 10.0)),
               RBF(length_scale=2.0, length_scale_bounds=(1e-1, 10.0))],
    'alpha': [1e-5, 1e-4, 1e-3],
    'optimizer': ['fmin_l_bfgs_b', 'fmin_ncg'],
    'n_restarts_optimizer': [0, 5, 10]
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Gaussian Process Regression model
    gpr_model = GaussianProcessRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(gpr_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_gpr_model = grid_search.best_estimator_

    # Fit the best model on the training fold
    best_gpr_model.fit(X_train_fold, y_train_fold)

    # Evaluate the best model on the validation set
    y_pred_val = best_gpr_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        gpr_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        gpr_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        gpr_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        gpr_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        gpr_rmse_list.append(gpr_rmse_val)
        gpr_mae_list.append(gpr_mae_val)
        gpr_r2_list.append(gpr_r2_val)
        gpr_explained_variance_list.append(gpr_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': gpr_rmse_val,
            'MAE': gpr_mae_val,
            'R2 Score': gpr_r2_val,
            'Explained Variance': gpr_explained_variance_val
        }, ignore_index=True)

        print("GPR RMSE for", target_variable, ":", gpr_rmse_val)
        print("GPR MAE for", target_variable, ":", gpr_mae_val)
        print("GPR R2 score for", target_variable, ":", gpr_r2_val)
        print("GPR Explained variance score for", target_variable, ":", gpr_explained_variance_val)



C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

GPR RMSE for p20_1200LR : 0.027748161736646265
GPR MAE for p20_1200LR : 0.022888670949259574
GPR R2 score for p20_1200LR : 0.9146811182537331
GPR Explained variance score for p20_1200LR : 0.9260595564031792
GPR RMSE for p20_1200LIB : 0.02774816177179446
GPR MAE for p20_1200LIB : 0.02288867099678431
GPR R2 score for p20_1200LIB : 0.9146811178266069
GPR Explained variance score for p20_1200LIB : 0.9260595559639754
GPR RMSE for P20_1200DT : 0.027429417970178033
GPR MAE for P20_1200DT : 0.02199563990960586
GPR R2 score for P20_1200DT : 0.9291049214886942
GPR Explained variance score for P20_1200DT : 0.9370734871477299
GPR RMSE for P20_1200ridge : 0.028779937890094753
GPR MAE for P20_1200ridge : 0.022928818065328316
GPR R2 score for P20_1200ridge : 0.9374501246320643
GPR Explained variance score for P20_1200ridge : 0.9417458243560453
GPR RMSE for Predicted_p20_1200knn : 0.02948676613716261
GPR MAE for Predicted_p20_1200knn : 0.024530829509166983
GPR R2 score for Predicted_p20_1200knn : 0.92

C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: Futu

GPR RMSE for p20_1200LR : 0.036804309449692166
GPR MAE for p20_1200LR : 0.023873940057849072
GPR R2 score for p20_1200LR : 0.9265640645320417
GPR Explained variance score for p20_1200LR : 0.9357373863234452
GPR RMSE for p20_1200LIB : 0.03680430948831512
GPR MAE for p20_1200LIB : 0.0238739401161458
GPR R2 score for p20_1200LIB : 0.9265640642266038
GPR Explained variance score for p20_1200LIB : 0.9357373859324248
GPR RMSE for P20_1200DT : 0.036193441981771386
GPR MAE for P20_1200DT : 0.024413486630776894
GPR R2 score for P20_1200DT : 0.9400005292514575
GPR Explained variance score for P20_1200DT : 0.9473906960261983
GPR RMSE for P20_1200ridge : 0.037212522424603316
GPR MAE for P20_1200ridge : 0.02554837406316369
GPR R2 score for P20_1200ridge : 0.9492523691438879
GPR Explained variance score for P20_1200ridge : 0.9551132793949286
GPR RMSE for Predicted_p20_1200knn : 0.042311755406593016
GPR MAE for Predicted_p20_1200knn : 0.026885382844521202
GPR R2 score for Predicted_p20_1200knn : 0.93

...

GPR RMSE for p20_1200LR : 0.07272235596943122
GPR MAE for p20_1200LR : 0.03786342795663576
GPR R2 score for p20_1200LR : 0.8820263678794001
GPR Explained variance score for p20_1200LR : 0.891156814360742
GPR RMSE for p20_1200LIB : 0.0727223564280518
GPR MAE for p20_1200LIB : 0.03786342801742953
GPR R2 score for p20_1200LIB : 0.882026366218797
GPR Explained variance score for p20_1200LIB : 0.8911568128947991
GPR RMSE for P20_1200DT : 0.05018798261644356
GPR MAE for P20_1200DT : 0.03152966340024344
GPR R2 score for P20_1200DT : 0.946360515886535
GPR Explained variance score for P20_1200DT : 0.9507861098464159
GPR RMSE for P20_1200ridge : 0.0353135629986072
GPR MAE for P20_1200ridge : 0.025080720288869703
GPR R2 score for P20_1200ridge : 0.9755972211050931
GPR Explained variance score for P20_1200ridge : 0.9764293172188075
GPR RMSE for Predicted_p20_1200knn : 0.04591067993411411
GPR MAE for Predicted_p20_1200knn : 0.0325647494869949
GPR R2 score for Predicted_p20_1200knn : 0.9603016534094

C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

GPR RMSE for p20_1200LR : 0.028409699881722865
GPR MAE for p20_1200LR : 0.02071373043477531
GPR R2 score for p20_1200LR : 0.9295436075175186
GPR Explained variance score for p20_1200LR : 0.929605944679949
GPR RMSE for p20_1200LIB : 0.028409699966675105
GPR MAE for p20_1200LIB : 0.02071373044364292
GPR R2 score for p20_1200LIB : 0.9295436070236966
GPR Explained variance score for p20_1200LIB : 0.9296059442040041
GPR RMSE for P20_1200DT : 0.027944559129766632
GPR MAE for P20_1200DT : 0.019302473549731792
GPR R2 score for P20_1200DT : 0.9474537384901913
GPR Explained variance score for P20_1200DT : 0.9474966457584257
GPR RMSE for P20_1200ridge : 0.028171319785030593
GPR MAE for P20_1200ridge : 0.01733372055535638
GPR R2 score for P20_1200ridge : 0.9608151142389987
GPR Explained variance score for P20_1200ridge : 0.960833580471207
GPR RMSE for Predicted_p20_1200knn : 0.03367612936429671
GPR MAE for Predicted_p20_1200knn : 0.023508028217591848
GPR R2 score for Predicted_p20_1200knn : 0.9325

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2108017082.py:64: Futu

In [52]:
# Calculate the mean errors for each target variable
mean_rmse = np.mean(gpr_rmse_list)
mean_mae = np.mean(gpr_mae_list)
mean_r2 = np.mean(gpr_r2_list)
mean_explained_variance = np.mean(gpr_explained_variance_list)

print("\nMean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 score:", mean_r2)
print("Mean Explained variance score:", mean_explained_variance)

# Print the error DataFrame in a tabular format
print(error_df)



Mean RMSE: 0.03407525512793342
Mean MAE: 0.024090129695817343
Mean R2 score: 0.9509504332588348
Mean Explained variance score: 0.9536124276888448
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.027748  0.022889  0.914681            0.926060
1             p20_1200LIB  0.027748  0.022889  0.914681            0.926060
2              P20_1200DT  0.027429  0.021996  0.929105            0.937073
3           P20_1200ridge  0.028780  0.022929  0.937450            0.941746
4   Predicted_p20_1200knn  0.029487  0.024531  0.926130            0.936543
..                    ...       ...       ...       ...                 ...
65             P20_1200DT  0.027945  0.019302  0.947454            0.947497
66          P20_1200ridge  0.028171  0.017334  0.960815            0.960834
67  Predicted_p20_1200knn  0.033676  0.023508  0.932576            0.932616
68                   ANN2  0.028340  0.017303  0.961890            0.961908
69               

In [53]:
y_pred_test = best_model.predict(X_test)
# Create a DataFrame to store the evaluation metrics
evaluation_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the evaluation metrics for all target variables
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test, multioutput='raw_values'))
mae_test = mean_absolute_error(y_test, y_pred_test, multioutput='raw_values')
r2_test = r2_score(y_test, y_pred_test, multioutput='raw_values')
explained_variance_test = explained_variance_score(y_test, y_pred_test, multioutput='raw_values')

# Print the evaluation metrics for all target variables
for i, target_variable in enumerate(target_variables):
    print("Linear Regression Test RMSE for", target_variable, ":", rmse_test[i])
    print("Linear Regression Test MAE for", target_variable, ":", mae_test[i])
    print("Linear Regression Test R2 Score for", target_variable, ":", r2_test[i])
    print("Linear Regression Test Explained Variance for", target_variable, ":", explained_variance_test[i])
    print()

# Calculate the mean of all errors
mean_rmse = np.mean(rmse_test)
mean_mae = np.mean(mae_test)
mean_r2 = np.mean(r2_test)
mean_explained_variance = np.mean(explained_variance_test)



Linear Regression Test RMSE for p20_1200LR : 0.0554652914719766
Linear Regression Test MAE for p20_1200LR : 0.036338813583912014
Linear Regression Test R2 Score for p20_1200LR : 0.9081962146211129
Linear Regression Test Explained Variance for p20_1200LR : 0.9098992417710359

Linear Regression Test RMSE for p20_1200LIB : 0.05546529137259941
Linear Regression Test MAE for p20_1200LIB : 0.036338813515800816
Linear Regression Test R2 Score for p20_1200LIB : 0.9081962147468668
Linear Regression Test Explained Variance for p20_1200LIB : 0.909899241913179

Linear Regression Test RMSE for P20_1200DT : 0.05260983407542856
Linear Regression Test MAE for P20_1200DT : 0.03459280974619316
Linear Regression Test R2 Score for P20_1200DT : 0.921738208657457
Linear Regression Test Explained Variance for P20_1200DT : 0.9230337297398824

Linear Regression Test RMSE for P20_1200ridge : 0.051773929327021594
Linear Regression Test MAE for P20_1200ridge : 0.03425496713803745
Linear Regression Test R2 Score f

In [54]:
# Print the mean errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.0552553398425179
Mean MAE: 0.036349356358412424
Mean R2 Score: 0.9182233992744895
Mean Explained Variance: 0.9196402681245772


In [55]:
# Test the Lasso model
y_pred_lasso = best_lasso_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_lasso, multioutput='raw_values'))
lasso_mae_test = mean_absolute_error(y_test, y_pred_lasso, multioutput='raw_values')
lasso_r2_test = r2_score(y_test, y_pred_lasso, multioutput='raw_values')
lasso_explained_variance_test = explained_variance_score(y_test, y_pred_lasso, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Lasso Test RMSE for", target_variable, ":", lasso_rmse_test[i])
    print("Lasso Test MAE for", target_variable, ":", lasso_mae_test[i])
    print("Lasso Test R2 Score for", target_variable, ":", lasso_r2_test[i])
    print("Lasso Test Explained Variance for", target_variable, ":", lasso_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': lasso_rmse_test[i],
        'MAE': lasso_mae_test[i],
        'R2 Score': lasso_r2_test[i],
        'Explained Variance': lasso_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


Lasso Test RMSE for p20_1200LR : 0.17406746836823153
Lasso Test MAE for p20_1200LR : 0.11874112380429958
Lasso Test R2 Score for p20_1200LR : 0.09582349885124619
Lasso Test Explained Variance for p20_1200LR : 0.13163943958190172

Lasso Test RMSE for p20_1200LIB : 0.1740674682379356
Lasso Test MAE for p20_1200LIB : 0.11874112373634745
Lasso Test R2 Score for p20_1200LIB : 0.0958234982033902
Lasso Test Explained Variance for p20_1200LIB : 0.13163943904279418

Lasso Test RMSE for P20_1200DT : 0.17192763071505815
Lasso Test MAE for P20_1200DT : 0.12205204394290485
Lasso Test R2 Score for P20_1200DT : 0.1641905507142798
Lasso Test Explained Variance for P20_1200DT : 0.1992747608476031

Lasso Test RMSE for P20_1200ridge : 0.17127815087552845
Lasso Test MAE for P20_1200ridge : 0.1270069203320603
Lasso Test R2 Score for P20_1200ridge : 0.24962696843565135
Lasso Test Explained Variance for P20_1200ridge : 0.28378304049084535

Lasso Test RMSE for Predicted_p20_1200knn : 0.18689612823549295
Lasso

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3630861469.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3630861469.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3630861469.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\3630861469.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [56]:
# Calculate the average of all errors
mean_rmse = np.mean(lasso_rmse_test)
mean_mae = np.mean(lasso_mae_test)
mean_r2 = np.mean(lasso_r2_test)
mean_explained_variance = np.mean(lasso_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.1743810830442514
Mean MAE: 0.12417332190104592
Mean R2 Score: 0.1876384121436449
Mean Explained Variance: 0.22289526711609625


In [57]:
# Test the Ridge model
y_pred_ridge = best_ridge_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ridge_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ridge, multioutput='raw_values'))
ridge_mae_test = mean_absolute_error(y_test, y_pred_ridge, multioutput='raw_values')
ridge_r2_test = r2_score(y_test, y_pred_ridge, multioutput='raw_values')
ridge_explained_variance_test = explained_variance_score(y_test, y_pred_ridge, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Ridge Test RMSE for", target_variable, ":", ridge_rmse_test[i])
    print("Ridge Test MAE for", target_variable, ":", ridge_mae_test[i])
    print("Ridge Test R2 Score for", target_variable, ":", ridge_r2_test[i])
    print("Ridge Test Explained Variance for", target_variable, ":", ridge_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ridge_rmse_test[i],
        'MAE': ridge_mae_test[i],
        'R2 Score': ridge_r2_test[i],
        'Explained Variance': ridge_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


Ridge Test RMSE for p20_1200LR : 0.05555965306997172
Ridge Test MAE for p20_1200LR : 0.036381104907362516
Ridge Test R2 Score for p20_1200LR : 0.9078835823239473
Ridge Test Explained Variance for p20_1200LR : 0.9096129426862937

Ridge Test RMSE for p20_1200LIB : 0.05555639773415688
Ridge Test MAE for p20_1200LIB : 0.03638022711189128
Ridge Test R2 Score for p20_1200LIB : 0.9078943763231254
Ridge Test Explained Variance for p20_1200LIB : 0.9096239032883779

Ridge Test RMSE for P20_1200DT : 0.05268012609665589
Ridge Test MAE for P20_1200DT : 0.03463329321514876
Ridge Test R2 Score for P20_1200DT : 0.9215289377232604
Ridge Test Explained Variance for P20_1200DT : 0.9228483918233953

Ridge Test RMSE for P20_1200ridge : 0.05181164963776606
Ridge Test MAE for P20_1200ridge : 0.034293370836560554
Ridge Test R2 Score for P20_1200ridge : 0.9313361539312746
Ridge Test Explained Variance for P20_1200ridge : 0.9322952483981475

Ridge Test RMSE for Predicted_p20_1200knn : 0.06804078590307518
Ridge 

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2441406475.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2441406475.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2441406475.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2441406475.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [58]:
# Calculate the average of all errors
mean_rmse = np.mean(ridge_rmse_test)
mean_mae = np.mean(ridge_mae_test)
mean_r2 = np.mean(ridge_r2_test)
mean_explained_variance = np.mean(ridge_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.05531857733637907
Mean MAE: 0.036382853258323626
Mean R2 Score: 0.9180300861051006
Mean Explained Variance: 0.9194725640723896


In [59]:
# Test the KNN model
y_pred_knn = best_knn_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
knn_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_knn, multioutput='raw_values'))
knn_mae_test = mean_absolute_error(y_test, y_pred_knn, multioutput='raw_values')
knn_r2_test = r2_score(y_test, y_pred_knn, multioutput='raw_values')
knn_explained_variance_test = explained_variance_score(y_test, y_pred_knn, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("KNN Test RMSE for", target_variable, ":", knn_rmse_test[i])
    print("KNN Test MAE for", target_variable, ":", knn_mae_test[i])
    print("KNN Test R2 Score for", target_variable, ":", knn_r2_test[i])
    print("KNN Test Explained Variance for", target_variable, ":", knn_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': knn_rmse_test[i],
        'MAE': knn_mae_test[i],
        'R2 Score': knn_r2_test[i],
        'Explained Variance': knn_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


KNN Test RMSE for p20_1200LR : 0.06878245384746831
KNN Test MAE for p20_1200LR : 0.035265937134954156
KNN Test R2 Score for p20_1200LR : 0.8588199534600334
KNN Test Explained Variance for p20_1200LR : 0.8673728866669738

KNN Test RMSE for p20_1200LIB : 0.06878245372676375
KNN Test MAE for p20_1200LIB : 0.03526593708104496
KNN Test R2 Score for p20_1200LIB : 0.8588199536430253
KNN Test Explained Variance for p20_1200LIB : 0.8673728868895808

KNN Test RMSE for P20_1200DT : 0.06697980091902751
KNN Test MAE for P20_1200DT : 0.03597955080907062
KNN Test R2 Score for P20_1200DT : 0.8731461580886553
KNN Test Explained Variance for P20_1200DT : 0.8807742434757476

KNN Test RMSE for P20_1200ridge : 0.06622148136727485
KNN Test MAE for P20_1200ridge : 0.03761679651551608
KNN Test R2 Score for P20_1200ridge : 0.8878314594851983
KNN Test Explained Variance for P20_1200ridge : 0.8947641484469288

KNN Test RMSE for Predicted_p20_1200knn : 0.08185017313657146
KNN Test MAE for Predicted_p20_1200knn : 

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1128329651.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1128329651.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1128329651.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1128329651.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [60]:
# Calculate the average of all errors
mean_rmse = np.mean(knn_rmse_test)
mean_mae = np.mean(knn_mae_test)
mean_r2 = np.mean(knn_r2_test)
mean_explained_variance = np.mean(knn_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.06929073442215934
Mean MAE: 0.037440784526985085
Mean R2 Score: 0.8717410089970257
Mean Explained Variance: 0.8798227227747899


In [61]:
# Test the ANN model
y_pred_ann = best_ann_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ann_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ann, multioutput='raw_values'))
ann_mae_test = mean_absolute_error(y_test, y_pred_ann, multioutput='raw_values')
ann_r2_test = r2_score(y_test, y_pred_ann, multioutput='raw_values')
ann_explained_variance_test = explained_variance_score(y_test, y_pred_ann, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("ANN Test RMSE for", target_variable, ":", ann_rmse_test[i])
    print("ANN Test MAE for", target_variable, ":", ann_mae_test[i])
    print("ANN Test R2 Score for", target_variable, ":", ann_r2_test[i])
    print("ANN Test Explained Variance for", target_variable, ":", ann_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ann_rmse_test[i],
        'MAE': ann_mae_test[i],
        'R2 Score': ann_r2_test[i],
        'Explained Variance': ann_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


ANN Test RMSE for p20_1200LR : 0.04841131019114858
ANN Test MAE for p20_1200LR : 0.03384579865671486
ANN Test R2 Score for p20_1200LR : 0.9300622489652454
ANN Test Explained Variance for p20_1200LR : 0.9321794803608411

ANN Test RMSE for p20_1200LIB : 0.051575734620818985
ANN Test MAE for p20_1200LIB : 0.036613158293267396
ANN Test R2 Score for p20_1200LIB : 0.9206204128216187
ANN Test Explained Variance for p20_1200LIB : 0.9247002962525753

ANN Test RMSE for P20_1200DT : 0.06141424652387834
ANN Test MAE for P20_1200DT : 0.040456112259105455
ANN Test R2 Score for P20_1200DT : 0.8933516413193001
ANN Test Explained Variance for P20_1200DT : 0.893851984525232

ANN Test RMSE for P20_1200ridge : 0.05633654008355738
ANN Test MAE for P20_1200ridge : 0.04405356826932021
ANN Test R2 Score for P20_1200ridge : 0.918819143654935
ANN Test Explained Variance for P20_1200ridge : 0.9397163108273839

ANN Test RMSE for Predicted_p20_1200knn : 0.061248682553222765
ANN Test MAE for Predicted_p20_1200knn :

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\230217354.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\230217354.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\230217354.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\230217354.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ip

In [62]:
# Calculate the average of all errors
mean_rmse = np.mean(ann_rmse_test)
mean_mae = np.mean(ann_mae_test)
mean_r2 = np.mean(ann_r2_test)
mean_explained_variance = np.mean(ann_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.05938483201889545
Mean MAE: 0.04117763735860097
Mean R2 Score: 0.9029436429085375
Mean Explained Variance: 0.9073014691441518


In [63]:
# Test the Elastic Net model
y_pred_en = best_en_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
en_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_en, multioutput='raw_values'))
en_mae_test = mean_absolute_error(y_test, y_pred_en, multioutput='raw_values')
en_r2_test = r2_score(y_test, y_pred_en, multioutput='raw_values')
en_explained_variance_test = explained_variance_score(y_test, y_pred_en, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Elastic Net Test RMSE for", target_variable, ":", en_rmse_test[i])
    print("Elastic Net Test MAE for", target_variable, ":", en_mae_test[i])
    print("Elastic Net Test R2 Score for", target_variable, ":", en_r2_test[i])
    print("Elastic Net Test Explained Variance for", target_variable, ":", en_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': en_rmse_test[i],
        'MAE': en_mae_test[i],
        'R2 Score': en_r2_test[i],
        'Explained Variance': en_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


Elastic Net Test RMSE for p20_1200LR : 0.10916000449624286
Elastic Net Test MAE for p20_1200LR : 0.06693027361213541
Elastic Net Test R2 Score for p20_1200LR : 0.6444136512989069
Elastic Net Test Explained Variance for p20_1200LR : 0.6591885554165229

Elastic Net Test RMSE for p20_1200LIB : 0.10916000438837031
Elastic Net Test MAE for p20_1200LIB : 0.06693027356578797
Elastic Net Test R2 Score for p20_1200LIB : 0.64441365121457
Elastic Net Test Explained Variance for p20_1200LIB : 0.6591885553936468

Elastic Net Test RMSE for P20_1200DT : 0.10464055635474288
Elastic Net Test MAE for P20_1200DT : 0.06746869418510523
Elastic Net Test R2 Score for P20_1200DT : 0.690389272247993
Elastic Net Test Explained Variance for P20_1200DT : 0.7046258067649886

Elastic Net Test RMSE for P20_1200ridge : 0.10129013621954358
Elastic Net Test MAE for P20_1200ridge : 0.06933208090495717
Elastic Net Test R2 Score for P20_1200ridge : 0.7375735556475725
Elastic Net Test Explained Variance for P20_1200ridge :

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2736310358.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2736310358.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2736310358.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2736310358.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [64]:
# Calculate the average of all errors
mean_rmse = np.mean(en_rmse_test)
mean_mae = np.mean(en_mae_test)
mean_r2 = np.mean(en_r2_test)
mean_explained_variance = np.mean(en_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.1066045779413967
Mean MAE: 0.06933144882553921
Mean R2 Score: 0.6954505732824515
Mean Explained Variance: 0.7099663385681646


In [65]:
# Test the Decision Tree model
y_pred_dt = best_dt_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
dt_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_dt, multioutput='raw_values'))
dt_mae_test = mean_absolute_error(y_test, y_pred_dt, multioutput='raw_values')
dt_r2_test = r2_score(y_test, y_pred_dt, multioutput='raw_values')
dt_explained_variance_test = explained_variance_score(y_test, y_pred_dt, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Decision Tree Test RMSE for", target_variable, ":", dt_rmse_test[i])
    print("Decision Tree Test MAE for", target_variable, ":", dt_mae_test[i])
    print("Decision Tree Test R2 Score for", target_variable, ":", dt_r2_test[i])
    print("Decision Tree Test Explained Variance for", target_variable, ":", dt_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': dt_rmse_test[i],
        'MAE': dt_mae_test[i],
        'R2 Score': dt_r2_test[i],
        'Explained Variance': dt_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


Decision Tree Test RMSE for p20_1200LR : 0.07807568315000837
Decision Tree Test MAE for p20_1200LR : 0.04346156446271521
Decision Tree Test R2 Score for p20_1200LR : 0.8180929312392228
Decision Tree Test Explained Variance for p20_1200LR : 0.8292506564301986

Decision Tree Test RMSE for p20_1200LIB : 0.07807568303723414
Decision Tree Test MAE for p20_1200LIB : 0.04346156442916145
Decision Tree Test R2 Score for p20_1200LIB : 0.8180929313620564
Decision Tree Test Explained Variance for p20_1200LIB : 0.8292506565967356

Decision Tree Test RMSE for P20_1200DT : 0.07497735895622366
Decision Tree Test MAE for P20_1200DT : 0.043164914840137535
Decision Tree Test R2 Score for P20_1200DT : 0.8410442666028156
Decision Tree Test Explained Variance for P20_1200DT : 0.8522426098528721

Decision Tree Test RMSE for P20_1200ridge : 0.073214431230998
Decision Tree Test MAE for P20_1200ridge : 0.04522614367392951
Decision Tree Test R2 Score for P20_1200ridge : 0.8628907764082063
Decision Tree Test Expl

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2533229606.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2533229606.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2533229606.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\2533229606.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [66]:
# Calculate the average of all errors
mean_rmse = np.mean(dt_rmse_test)
mean_mae = np.mean(dt_mae_test)
mean_r2 = np.mean(dt_r2_test)
mean_explained_variance = np.mean(dt_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.07626871466537857
Mean MAE: 0.04484044006415249
Mean R2 Score: 0.8441668504676915
Mean Explained Variance: 0.8556741926611169


In [67]:
# Test the RANSAC model
y_pred_ransac = best_ransac_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ransac_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ransac, multioutput='raw_values'))
ransac_mae_test = mean_absolute_error(y_test, y_pred_ransac, multioutput='raw_values')
ransac_r2_test = r2_score(y_test, y_pred_ransac, multioutput='raw_values')
ransac_explained_variance_test = explained_variance_score(y_test, y_pred_ransac, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("RANSAC Test RMSE for", target_variable, ":", ransac_rmse_test[i])
    print("RANSAC Test MAE for", target_variable, ":", ransac_mae_test[i])
    print("RANSAC Test R2 Score for", target_variable, ":", ransac_r2_test[i])
    print("RANSAC Test Explained Variance for", target_variable, ":", ransac_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ransac_rmse_test[i],
        'MAE': ransac_mae_test[i],
        'R2 Score': ransac_r2_test[i],
        'Explained Variance': ransac_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


RANSAC Test RMSE for p20_1200LR : 0.0556029069544811
RANSAC Test MAE for p20_1200LR : 0.03643468697147718
RANSAC Test R2 Score for p20_1200LR : 0.9077400989283113
RANSAC Test Explained Variance for p20_1200LR : 0.9100043475489731

RANSAC Test RMSE for p20_1200LIB : 0.05560290685959638
RANSAC Test MAE for p20_1200LIB : 0.03643468690638431
RANSAC Test R2 Score for p20_1200LIB : 0.9077400990389634
RANSAC Test Explained Variance for p20_1200LIB : 0.9100043476737839



C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({


RANSAC Test RMSE for P20_1200DT : 0.05271141648035709
RANSAC Test MAE for P20_1200DT : 0.034781232592830176
RANSAC Test R2 Score for P20_1200DT : 0.921435691216722
RANSAC Test Explained Variance for P20_1200DT : 0.9231975655572842

RANSAC Test RMSE for P20_1200ridge : 0.0518272278840196
RANSAC Test MAE for P20_1200ridge : 0.03449010558180985
RANSAC Test R2 Score for P20_1200ridge : 0.9312948573071244
RANSAC Test Explained Variance for P20_1200ridge : 0.9325953054703747

RANSAC Test RMSE for Predicted_p20_1200knn : 0.06855671686052758
RANSAC Test MAE for Predicted_p20_1200knn : 0.04459640126338199
RANSAC Test R2 Score for Predicted_p20_1200knn : 0.892737436780391
RANSAC Test Explained Variance for Predicted_p20_1200knn : 0.8961460341307207

RANSAC Test RMSE for ANN2 : 0.05174299654677446
RANSAC Test MAE for ANN2 : 0.03455375784616959
RANSAC Test R2 Score for ANN2 : 0.9323804799300565
RANSAC Test Explained Variance for ANN2 : 0.9336147448528446

RANSAC Test RMSE for en : 0.05188188086770

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1601085822.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [68]:

# Calculate the average of all errors
mean_rmse = np.mean(ransac_rmse_test)
mean_mae = np.mean(ransac_mae_test)
mean_r2 = np.mean(ransac_r2_test)
mean_explained_variance = np.mean(ransac_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.055418007493351906
Mean MAE: 0.036536959517526134
Mean R2 Score: 0.9177222220348822
Mean Explained Variance: 0.9196601596815065


In [69]:
# Test the GPR model
y_pred_gpr = best_gpr_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
gpr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_gpr, multioutput='raw_values'))
gpr_mae_test = mean_absolute_error(y_test, y_pred_gpr, multioutput='raw_values')
gpr_r2_test = r2_score(y_test, y_pred_gpr, multioutput='raw_values')
gpr_explained_variance_test = explained_variance_score(y_test, y_pred_gpr, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("GPR Test RMSE for", target_variable, ":", gpr_rmse_test[i])
    print("GPR Test MAE for", target_variable, ":", gpr_mae_test[i])
    print("GPR Test R2 Score for", target_variable, ":", gpr_r2_test[i])
    print("GPR Test Explained Variance for", target_variable, ":", gpr_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': gpr_rmse_test[i],
        'MAE': gpr_mae_test[i],
        'R2 Score': gpr_r2_test[i],
        'Explained Variance': gpr_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


GPR Test RMSE for p20_1200LR : 0.04460931834145759
GPR Test MAE for p20_1200LR : 0.029921035296960132
GPR Test R2 Score for p20_1200LR : 0.9406160387895979
GPR Test Explained Variance for p20_1200LR : 0.941101618981084

GPR Test RMSE for p20_1200LIB : 0.04460931820030057
GPR Test MAE for p20_1200LIB : 0.029921035168365803
GPR Test R2 Score for p20_1200LIB : 0.9406160390339628
GPR Test Explained Variance for p20_1200LIB : 0.9411016192260576

GPR Test RMSE for P20_1200DT : 0.04189772275916925
GPR Test MAE for P20_1200DT : 0.028909876693381735
GPR Test R2 Score for P20_1200DT : 0.9503640021075708
GPR Test Explained Variance for P20_1200DT : 0.9508104717307486

GPR Test RMSE for P20_1200ridge : 0.040966880198823816
GPR Test MAE for P20_1200ridge : 0.029044486081398264
GPR Test R2 Score for P20_1200ridge : 0.9570721567530125
GPR Test Explained Variance for P20_1200ridge : 0.9575607384943767

GPR Test RMSE for Predicted_p20_1200knn : 0.055184964066526246
GPR Test MAE for Predicted_p20_1200kn

C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1759785292.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1759785292.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1759785292.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Temp\ipykernel_17584\1759785292.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
C:\Users\kirth\AppData\Local\Tem

In [70]:
# Calculate the average of all errors
mean_rmse = np.mean(gpr_rmse_test)
mean_mae = np.mean(gpr_mae_test)
mean_r2 = np.mean(gpr_r2_test)
mean_explained_variance = np.mean(gpr_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.04415678956667224
Mean MAE: 0.030302129278683226
Mean R2 Score: 0.9476859782491919
Mean Explained Variance: 0.9483121244083099
